# ¿De que forma es más eficiente decir tu DNI?
> Sebastian Andres : https://linktr.ee/sebasandres.idd

La idea es evaluar la eficiencia de distintos patrones para decir cualquier DNI (*hay algunas limitaciones*), es decir ¿De qué forma agrupo los numeros para expresar mi numero de documento?

Para esto, voy a suponer (puede no serlo) que la eficiencia de una frase es directamente proporcional a su contenido e inversamente proporcional a la cantidad de letras\caracteres que contiene.

Entonces, cuanto menor longitud de caracteres tenga la frase que pronuncia el dni, mas eficiente es bajo esta perspectiva.

Luego, voy a hacer un algoritmo que dado un dni explore todas los patrones posibles (XX-X-XX-X-XX, ...) y devuelva los mas eficientes para ese caso.


## Idea del Algoritmo:

Como inputs va a tener el DNI y n_depth, un numero que representa el nivel de profunidad (cantidad de niveles con resultados de igual eficiencia) que se busca obtener.

El output es una lista de niveles. Cada nivel a su vez es una lista con patrones de igual eficiencia. 
 
    def GET_BESTS_PATTERNS (dni, n_depth) {
        vectDni = vectorizeDni(dni) # paso el dni de int a list
        PatronesAProbar = PermutarPatrones(len(vectDni)) # armo una lista con todas las permutaciones posibles
        
        Patterns = [] # armo una lista de objetos con los patrones, su pronunciacion y eficacia
        for patron in PatronesAProbar:
            pron = PronunciarPorPatron(dni, patron)
            Patterns.append(Pattern(patron, pron, len(pron)))
        
        OrderedPatterns = OrderPatterns (Patterns)
        Levels = GET_N_LEVELS_DEPTH(OrderedPatterns, n_depth)
        return Levels
    }
        
A continuacion la explicacion de cada uno de los sub-algoritmos usados en esta funcion:

### 1) Modulo Pronunciar: ¿Cómo pasar de un numero en INT a STRING?

In [10]:
## Primero armo un modulo que pueda contar --> Practico pero muy mejorable 

numeros_unidad = ["", "uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "diez"]
numeros_decena = ["e", "veinti", "treinti", "cuarenti", "cincuenti", "sesenti", "setenti", "ochenti", "noventi"]
numeros_centena = ["ciento", "doscientos", "trescientos", "cuatrocientos", "quinientos", "seiscientos", "setecientos", "ochocientos", "novecientos"]
# casos excepcion
decenas_perfectas = ["diez", "veinte", "treinta", "cuarenta", "cincuenta", "sesenta", "setenta", "ochenta", "noventa"]
decena_problematica = ["once", "doce", "trece", "catorce", "quince", "dieciseis", "diecisiete", "dieciocho", "diecinueve"]

def Pronunciar (numero): # Funcional para numeros entre (0, 10**9)
    pronunciacion = ""
    if 0 <= numero < 10**9:
        if numero < 10: # una cifra
            if numero == 0: pronunciacion = "cero"
            else: pronunciacion = numeros_unidad[numero]

        elif 10 <= numero < 100: # dos cifras
            if numero % 10 == 0:
                pronunciacion = decenas_perfectas [(numero // 10)-1]
            elif 11 <= numero <= 19:
                pronunciacion = decena_problematica[(numero % 10)-1]
            else: 
                digito2 = numero % 10
                digito1 = (numero - digito2) // 10
                pronunciacion = numeros_decena[digito1-1] + numeros_unidad[digito2]
        
        elif 100 <= numero < 1000: # tres cifras 
            if numero % 100 == 0:
                if numero == 100: pronunciacion = "cien"
                else: pronunciacion = numeros_centena[(numero//100)-1]
            else:
                pronunciacion = numeros_centena[(numero//100)-1] + Pronunciar(numero-(numero//100)*100)

        elif 1000 <= numero < 10**6: # cuatro a seis cifras
            if numero == 1000: pronunciacion = "mil"
            else:
                mil_i = numero // 1000
                if mil_i == 1:
                    pronunciacion = "mil" + Pronunciar (numero - mil_i*1000)
                else:
                    units = numero % (10**3)
                    if units == 0:
                        pronunciacion = Pronunciar (mil_i) + "mil"
                    else:
                        pronunciacion = Pronunciar (mil_i) + "mil" + Pronunciar (numero - mil_i*1000)

        elif 10**6 <= numero < 10**9: # siete cifras
            units = numero % (10**6)
            if units == 0:
                if numero == 10**6: pronunciacion = "un millon" 
                else:
                    pronunciacion = Pronunciar(numero // 10**6) + "millones" 
            else:
                pronunciacion = Pronunciar((numero // 10**6)*10**6) + Pronunciar (units)
    else:
        pronunciacion = "Fuera de rango"    
    
    return pronunciacion

Para evitar errores de pronunciacion (ej. Pronunciar("000") = cero), defino una funcion <b>strPronunciar</b> que atrapa estos digitos. Entonces uso strings para almacenar las listas de numeros.

In [2]:
def strPronunciar (strNumero):
    if len(strNumero) > 1:
        nCerosIzquierda = strNumero[:-1].count('0')
        if strNumero[0] == '0':
            return nCerosIzquierda * Pronunciar(0) + Pronunciar(int(strNumero[1:]))
    return Pronunciar(int(strNumero))

Podes probar la funcion <b>Pronununciar</b> en la siguiente celda:

In [5]:
def contar (desde, hasta):
    for numero in range(desde, hasta+1):
        print (numero, Pronunciar(numero))

print ("Contar")
desde = int(input("Desde: "))
hasta = int(input("Hasta: "))
contar (desde, hasta)

Contar
Desde: 0
Hasta: 5
0 cero
1 uno
2 dos
3 tres
4 cuatro
5 cinco


### Modulo PronunciarPorPatron: ¿Cómo paso de un patron como XX-XXX-XXX a una pronunciacion?

A este modulo le doy como input un dni y un patron y me devuelve la pronunciacion del dni siguiendo ese patron.

Funcionamiento: Armo una lista con el numero de items que tiene cada grupo. Armo los grupos con el dni provisto y concateno la pronunciacion de cada grupo.

In [7]:
def PronunciarPorPatron (dni, patron):
    pronunciacion = "" # inicializo vacia
    vectDni = vectorizeDni(dni) # paso de int --> list
    # cuento cuántos grupos me configura el patron y con cuantos caracteres c/u
    grupos_lengths = [] 
    n_chars_grupo = 0
    for ch in patron+"-":
        if ch == "-":
            grupos_lengths.append(n_chars_grupo)
            n_chars_grupo = 0
        else: 
            n_chars_grupo += 1
    # armo cada grupo con los caracteres del dni
    grupos = []
    init_index = 0
    for grupo_length in grupos_lengths:
        list_group = vectDni[init_index : init_index + grupo_length]
        number_group = strJoin (list_group)
        grupos.append (number_group)        
        init_index += grupo_length
    # agrego a pronunciacion la pronunciacion de cada grupo, separada por un guion
    for grupo in grupos:
        pronunciacion += strPronunciar(grupo) + "-"   
    return pronunciacion[:-1] # elimino el guion extra y devuelvo la pronunciacion

## funciones auxiliares

def vectorizeDni (dni):
    return list(str(dni)) 

def strJoin (listaNumeros_str):
    return "".join(listaNumeros_str)

### PermutarPatrones: ¿De donde saco todas las permutaciones posibles?

La idea es que un dni con M digitos habilita a $2^{M-1}$ patrones posibles. Pues existen M-1 barreras posibles entre cada digito, con 2 estados posibles. 

Entonces pienso a cada configuracion posible como un byte (o mejor dicho, lista de M-1 bits).

En particular, como $\forall i, \exists! byte : i \Re byte $ y busco relacionar biyectivamente $i$ con un patron, entonces uso al $byte$ como intermediario.

> XXXXXXXX --> X|X|X|X|X|X|X|X

Las barras (|) representan las M-1 barreras mencionadas, y solo estan presentes en la posicion j, por ejemplo, si solo si $byte[j]=1$.  

> Flujo: i (int) --> 01100.. (byte) --> XX-X-XXX... (patron)
         

In [6]:
def PermutarPatrones (nDigitos):
    # Devuelve todas las permutaciones de patrones posibles 
    # Obs. Hay 2**(nDigitos-1) permutaciones posibles
    # Agrego una barrera media entre cada numero y vario su estado
    # Para esto hago un loop desde i=0 a i=2^(n-1) y traduzco ese numero i en estados de los bits. 
    # Cada i esta relacionado biyectivamente con un patron
    patterns = list ()

    for i in range(2**(nDigitos-1)):
        # itero cada patron posible
        bits = format(i, "b") # lo paso a un str con los bits (ej."0101101")
        bits_list = [int(n) for n in list(bits)] # [0, 1, 0, 1, 1]
        bits_list = adaptToNBits (bits_list, nDigitos-1)
        # Creo el i_esimo patron siguiendo el bit
        patron_i = byte_to_patron_v1 (bits_list)
        patterns.append (patron_i)

    return patterns

def adaptToNBits (bits_list, n):
    ## Hacer que el bit tenga n digitos sin importar su tamaño
    ## Agregar ceros a la izquierda
    for n in range(n - len(bits_list)):
        bits_list.insert(0, 0)
    return bits_list

def byte_to_patron_v1 (byte):
    # v1 porque es la mas eficiente de las que probe
    patron_i = "X"
    for n in range (len(byte)):
        if byte[n] == 1:
            patron_i += "-X"
        else:
            patron_i += "X"    
    return patron_i

### OrderPatterns: Ordenar los patrones descendientemente con MergeSort

La idea es ir comparando las cabezas de ambas listas. Cuando el primer elemento de una de las listas sea menor (o igual), se agrega a la lista de retorno y se elimina de la lista. Asi hasta que una de las listas quede vacia.
Cuando esto suceda, la otra lista necesariamente va a ser mayor a cualquier elemento de la lista de retorno.

In [ ]:
def OrderPatterns (lst):
    if len(lst) == 1:
        return lst
    else:
        middle = len(lst) // 2
        left, right = lst[:middle], lst[middle:]
        left = OrderPatterns(left)
        right = OrderPatterns(right)
        if left[-1].score < right[0].score:
            left.extend(right)
            return left
        else:
            return merge(left, right)

def merge (left, right):
    ret = []
    while (len(left) != 0 and len(right) != 0):
        if (left[0].score <= right[0].score):
            ret.append(left[0])
            left.pop(0)
        else:
            ret.append(right[0])
            right.pop(0)
    # Lo restante deberia estar ordenado y ser mayor a todo elemento en ret
    if len(left) > 0:
        ret.extend(left)
    elif len(right) > 0:
        ret.extend(right)
    return ret

Para simplificar los indices cree un objeto Pattern

In [ ]:
class Pattern:
    def __init__ (self, _patr, _pron, _score):
        self.patr = _patr
        self.pron = _pron
        self.score = _score # len

Entonces, un MergeSort me queda una lista con patrones ordenados de la forma:

$Patrones = [(patr, pron, score)_0, (..., ..., ...)_1, ... , (patr, pron, score)_{2^{M-1}}]$

$Patrones = [Patron_0, Patron_1, ... , Patron_{2^{M-1}}]$

Con $Patrones_{j}.score <= Patrones_{i}.score <=> j <= i$

### Finalmente el algoritmo para obtener los n_depth niveles de profundidad 

Avanza en la lista hasta encontrar patrones con distinto score, aumentando el contador y terminando cuando este alcance n_depth.

    def GET_N_LEVELS_DEPTH (patterns, depth):
        j = 1
        levels = [[patterns_[0]]]
        for n -> n_depth-1 {
            while (patterns[j].score == last(levels)[0].score) {
                last(levels).append(patterns[j])
                if j < len(patterns)-1 
                    j++
                else
                    break
            }
            if len(levels) < n_depth:
                levels.append([patterns[j]])
                j ++
            else: break  
        }
        return levels

In [9]:
def GET_N_LEVELS_DEPTH (Patterns, n_depth):
    j = 1
    Levels = [[Patterns[0]]]
    for _ in range (n_depth):
        while ((Patterns[j].score == Levels[-1][0].score)):
            Levels[-1].append(Patterns[j])
            if (j < len (Patterns)-1):
                j += 1
            else:
                break
        if len(Levels) < n_depth:
            Levels.append([Patterns[j]])
            j += 1
        else: 
            break
    return Levels